# Demonstração de Lógica

Este documento apresenta uma visão geral da lógica inicial utilizada no projeto de animação 3D para rastreamento de trajetória da bola.

---


1. **Com a seguinte lógica não foi alcançada o objetio**  
   **Não foi possível concluir o trabalho** com a lógica inicialmente proposta.
   


2. **Problemas com o Arquivo CSV**  
   O arquivo CSV gerado **não refletiu a trajetória real da bola** conforme esperado.  



3. **Resultado da Animação 3D**  
   Ao realizar a animação 3D do arquivo CSV, os pontos exibidos correspondiam aos **pontos de calibração** em vez da trajetória da bola.


---

*Observação:* O arquivo que mostra a construção 3D da bola é -> **animação3D.py**

*Observação2:* Alterar os caminhos para que os **arquivos sejam lidos**

In [19]:
import cv2
import glob
import pandas as pd
import numpy as np

In [44]:
# Carrega as imagens em ordem
frames = sorted(glob.glob("caminho_das_imagens/*.png"))

# Configura o delay entre os frames (em milissegundos)
delay = 30  # Ajuste para alterar a velocidade do loop

for frame_path in frames:
    frame = cv2.imread(frame_path)
    cv2.imshow("Video", frame)
    if cv2.waitKey(delay) & 0xFF == ord('q'):  # Pressione 'q' para sair do loop
        break

cv2.destroyAllWindows()

In [33]:
# Lendo os arquivos
cp2d_c1 = pd.read_csv('C:/Users/55169/PycharmProjects/biomec_fut/Recostrução3DTarefa2/frames/c1cal/c1cal2rec3d.dat', delimiter=' ', header=None)
cp2d_c2 = pd.read_csv('C:/Users/55169/PycharmProjects/biomec_fut/Recostrução3DTarefa2/frames/c2cal/c2cal2rec3d.dat', delimiter=' ', header=None)
cp2dc1 = np.asarray(cp2d_c1)
cp2dc2 = np.asarray(cp2d_c2)

cp3d = pd.read_csv('ArquivosCSVTeste/ref3d_v2.ref', delimiter=' ', header=None)
cp3d = np.asarray(cp3d)

print(cp3d)

[[0.    0.    0.   ]
 [0.    1.466 0.   ]
 [0.781 1.466 0.   ]
 [0.781 0.    0.   ]
 [0.    0.    0.453]
 [0.    1.466 0.451]
 [0.781 1.466 0.447]
 [0.781 0.    0.45 ]
 [0.    0.    0.907]
 [0.    1.466 0.903]
 [0.781 1.466 0.902]
 [0.781 0.    0.903]]


In [26]:
cp3d = np.asarray(cp3d, dtype=float)
cp2d = np.asarray(cp2dc1, dtype=float)

m = np.size(cp3d[:, 0], 0)
M = np.zeros([m * 2, 11])
N = np.zeros([m * 2, 1])

for i in range(m):
    M[i*2,:] = [cp3d[i,0], cp3d[i,1], cp3d[i,2] ,1, 0, 0, 0, 0, -cp2d[i, 0] * cp3d[i, 0], -cp2d[i, 0] * cp3d[i, 1], -cp2d[i, 0] * cp3d[i, 2]]
    M[i*2+1,:] = [0 , 0, 0, 0, cp3d[i,0], cp3d[i,1], cp3d[i,2],1, -cp2d[i,1] * cp3d[i,0],-cp2d[i,1] * cp3d[i,1], -cp2d[i,1] * cp3d[i,2]]
    N[[i*2,i*2+1],0] = cp2d[i, :]

Mt = M.T
M1 = np.linalg.pinv(Mt.dot(M))
MN = Mt.dot(N)

DLT_c1 = (M1).dot(MN).T

print(DLT_c1)

[[-9.58777685e+01  1.53073599e+02 -9.09483052e+00  2.20036316e+02
   1.44604193e+01 -1.98849076e-01 -1.62701631e+02  1.62456869e+02
  -8.11031539e-02 -3.29078205e-02 -1.08117311e-02]]


In [27]:
cp3d = np.asarray(cp3d, dtype=float)
cp2d = np.asarray(cp2dc2, dtype=float)

m = np.size(cp3d[:, 0], 0)
M = np.zeros([m * 2, 11])
N = np.zeros([m * 2, 1])

for i in range(m):
    M[i*2,:] = [cp3d[i,0], cp3d[i,1], cp3d[i,2] ,1, 0, 0, 0, 0, -cp2d[i, 0] * cp3d[i, 0], -cp2d[i, 0] * cp3d[i, 1], -cp2d[i, 0] * cp3d[i, 2]]
    M[i*2+1,:] = [0 , 0, 0, 0, cp3d[i,0], cp3d[i,1], cp3d[i,2],1, -cp2d[i,1] * cp3d[i,0],-cp2d[i,1] * cp3d[i,1], -cp2d[i,1] * cp3d[i,2]]
    N[[i*2,i*2+1],0] = cp2d[i, :]

Mt = M.T
M1 = np.linalg.pinv(Mt.dot(M))
MN = Mt.dot(N)

DLT_c2 = (M1).dot(MN).T

print(DLT_c2)

[[ 3.76874916e+01  1.72851718e+02 -1.00674129e+01  1.92488518e+02
   1.14185135e+01 -9.45539638e+00 -1.59289522e+02  1.68360007e+02
  -9.33700937e-02  3.72583538e-02 -1.48028025e-02]]


In [37]:
def r3d(DLTs, cc2ds):
    # Montar sistema para triangulação a partir das duas visualizações DLT
    A = np.vstack([
        (DLTs[0, :3] - cc2ds[0, 0] * DLTs[0, 8:11]),
        (DLTs[0, 4:7] - cc2ds[0, 1] * DLTs[0, 8:11]),
        (DLTs[1, :3] - cc2ds[1, 0] * DLTs[1, 8:11]),
        (DLTs[1, 4:7] - cc2ds[1, 1] * DLTs[1, 8:11])
    ])
    B = np.array([
        cc2ds[0, 0] - DLTs[0, 3],
        cc2ds[0, 1] - DLTs[0, 7],
        cc2ds[1, 0] - DLTs[1, 3],
        cc2ds[1, 1] - DLTs[1, 7]
    ])

    # Resolver para obter coordenadas 3D
    coord_3d, _, _, _ = np.linalg.lstsq(A, B, rcond=None)
    return coord_3d


# Concatenação das calibrações DLT
DLTs = np.vstack([DLT_c1, DLT_c2])
print(DLTs)

[[-9.58777685e+01  1.53073599e+02 -9.09483052e+00  2.20036316e+02
   1.44604193e+01 -1.98849076e-01 -1.62701631e+02  1.62456869e+02
  -8.11031539e-02 -3.29078205e-02 -1.08117311e-02]
 [ 3.76874916e+01  1.72851718e+02 -1.00674129e+01  1.92488518e+02
   1.14185135e+01 -9.45539638e+00 -1.59289522e+02  1.68360007e+02
  -9.33700937e-02  3.72583538e-02 -1.48028025e-02]]


In [41]:
import sys

# Matriz para coordenadas 3D reconstruídas
cc3d = np.zeros((len(cp2dc1), 3))
for i in range(len(cp2dc1)):
    # Combinação dos pontos 2D de cada câmera para o ponto i
    cc2ds = np.vstack([cp2dc1[i, :], cp2dc2[i, :]])
    # Calcula as coordenadas 3D para o ponto i
    cc3d[i, :] = r3d(DLTs, cc2ds).T

# Salvamento das coordenadas 3D em um arquivo
output_file = sys.argv[4] if len(sys.argv) > 4 else "resultado_3d"
np.savetxt(output_file + '.3d', cc3d, fmt='%.10f')

ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

In [42]:
import pandas as pd

# Dados em formato de lista de listas (x, y, z)
data = [
    [0.0040106947, 0.0015723420, -0.0030787177],
    [0.0013577771, 1.4664854801, -0.0002735988],
    [0.7795888697, 1.4656860931, 0.0018643966],
    [0.7783916434, -0.0011725959, 0.0016910908],
    [-0.0015136300, -0.0004110298, 0.4560298090],
    [-0.0009480190, 1.4654112549, 0.4498028610],
    [0.7801485113, 1.4676494051, 0.4457739917],
    [0.7823768501, -0.0016201126, 0.4491580508],
    [0.0002223801, 0.0008604146, 0.9077355926],
    [-0.0032767218, 1.4643818640, 0.9040679840],
    [0.7859380844, 1.4664276938, 0.9016854732],
    [0.7797204633, 0.0007359349, 0.9015623193]
]

# Criar um DataFrame e definir colunas x, y, z
df = pd.DataFrame(data, columns=['x', 'y', 'z'])

# Salvar o DataFrame em um arquivo CSV
df.to_csv('resultado_3d.3d', index=False)